<span style="font-size: 14pt">MIPT, Applied ML, Autumn 2018</span>

<span style="font-size: 16pt"> HW #5: Approximate RL homework

<span style="color:red; font-size: 14pt;"> Дедлайн 5.12.2018 12:00 </span>

<span style="color:blue; font-size: 12pt">Valentin Malykh </span>,
<span style="color:blue; font-size: 12pt; font-family: 'Verdana'">val@maly.hk</span>


**Оформление дз**: 
- Выполненное задание требуется отправлять через <a href='https://goo.gl/forms/XPSIbwp7wPxB4SsI3'>форму </a>

- Выполненное дз прикрепляйте в формате файла ``<фамилия>_<группа>_task<номер>.ipynb``, например: ``ivanov_594_task4.ipynb`` 

**Вопросы**:
- Вопросы присылайте в канал в телеграмме ``[Fall 2018]ML Seminars``

--------
- **PS1**: Будьте внимательны при заполнении формы, когда отправляете ДЗ 


<h1 align="center">Практическое задание (100%)</h1>
Описание находится по ссылке: https://gist.github.com/madrugado/bbaedcb614755d47345c3c69b1b41169

In [1]:
import numpy as np
import tensorflow as tf
import random
import os
import gym
from gym import wrappers
from time import time
from datetime import datetime
from collections import deque

In [2]:
env = gym.make('BipedalWalker-v2')
#env = wrappers.Monitor(env, os.path.join('hw5', 'BipedalWalker-v2'), video_callable=None)

WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.
WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.


In [3]:
MAX_EPISODES = 1000  
LR_A = 0.0001  
LR_C = 0.001  
L2_DECAY = 0.01  
GAMMA = 0.99  
TAU = 0.001  

# Exploration noise parameters
OU_MU = 0.0
OU_THETA = 0.15  
OU_SIGMA = 0.20  
TAU2 = 25

BUFFER_SIZE = 1000000 
MINIBATCH_SIZE = 64

In [4]:
state_dim, action_dim = env.observation_space.shape[0], env.action_space.shape[0]
action_bound = (env.action_space.high - env.action_space.low) / 2

In [5]:
state_dim, action_dim, action_bound

(24, 4, array([1., 1., 1., 1.], dtype=float32))

In [6]:
config = tf.ConfigProto(log_device_placement=False, device_count={'GPU': 1})  
sess = tf.Session(config=config)
state = tf.placeholder(tf.float32, [None, state_dim], 'state')
action = tf.placeholder(tf.float32, [None, action_dim], 'action')
y_t = tf.placeholder(tf.float32, [None, action_dim], name="y_t")

In [7]:
def create_actor_network(name):
        with tf.variable_scope(name):
            l1 = tf.layers.dense(state, 400, tf.nn.relu, name='actor_L1')
            l2 = tf.layers.dense(l1, 400, tf.nn.relu, name='actor_L2')

            w_init = tf.random_uniform_initializer(minval=-0.003, maxval=0.003)
            action = tf.layers.dense(l2, action_dim, tf.nn.tanh, kernel_initializer=w_init, name='actor_output')
            scaled_action = tf.multiply(action, action_bound, name='actor_output_scaled')

        params = tf.get_collection(tf.GraphKeys.GLOBAL_VARIABLES, scope=name)
        return scaled_action, params

In [8]:
def create_critic_network(name):
        with tf.variable_scope(name):
            net = tf.layers.dense(state, 400, tf.nn.relu, name='critic_L1')
            initializer = tf.variance_scaling_initializer()
            s_union_weights = tf.Variable(initializer.__call__([400, 300]), name='critic_L2_Ws')
            a_union_weights = tf.Variable(initializer.__call__([action_dim, 300]), name='critic_L2_Wa')
            union_biases = tf.Variable(tf.zeros([300]), name='critic_L2_b')

            net = tf.nn.relu(tf.matmul(net, s_union_weights) +
                             tf.matmul(action, a_union_weights) + union_biases,
                             name='critic_L2')

            w_init = tf.random_uniform_initializer(minval=-0.003, maxval=0.003)
            value = tf.layers.dense(net, action_dim, kernel_initializer=w_init, name='critic_output')
        params = tf.get_collection(tf.GraphKeys.GLOBAL_VARIABLES, scope=name)
        return value, params

In [9]:
actor, actor_params = create_actor_network("mu")
actor_target, actor_target_params = create_actor_network("mu_target")
critic, critic_params = create_critic_network("q")
critic_target, critic_target_params = create_critic_network("q_target")

In [10]:
with tf.variable_scope('update_target'):
    update_actor = [pt.assign(p * TAU + pt * (1-TAU)) for p, pt in zip(actor_params, actor_target_params)]
    update_critic = [pt.assign(p * TAU + pt * (1-TAU)) for p, pt in zip(critic_params, critic_target_params)]

In [11]:
with tf.variable_scope('train'):
            loss_q = tf.losses.mean_squared_error(y_t, critic)
            tf.summary.scalar("loss_critic", loss_q)

            policy_gradient = tf.gradients(critic, action)[0]
            actor_gradient = tf.gradients(actor, actor_params, -policy_gradient)

            train_critic = tf.train.AdamOptimizer(LR_C).minimize(loss_q)
            train_actor = tf.train.AdamOptimizer(LR_A).apply_gradients(zip(actor_gradient, actor_params))

In [12]:
sess.run(tf.global_variables_initializer())

In [13]:
buffer = deque(maxlen=BUFFER_SIZE)
def sample_batch(batch_size):
        if len(buffer) < batch_size:
            batch = random.sample(buffer, len(buffer))
        else:
            batch = random.sample(buffer, batch_size)

        s_batch = np.array([_[0] for _ in batch])
        a_batch = np.array([_[1] for _ in batch])
        r_batch = np.array([_[2] for _ in batch]).reshape(batch_size, -1)
        t_batch = np.array([_[3] for _ in batch]).reshape(batch_size, -1)
        s2_batch = np.array([_[4] for _ in batch])
        return s_batch, a_batch, r_batch, t_batch, s2_batch

In [14]:
class OUNoise:
    def __init__(self, action_dimension, mu=0.0, theta=0.15, sigma=0.3, seed=123):
        self.action_dimension = action_dimension
        self.mu = mu
        self.theta = theta
        self.sigma = sigma
        self.state = np.ones(self.action_dimension) * self.mu
        self.reset()
        np.random.seed(seed)

    def reset(self):
        self.state = np.ones(self.action_dimension) * self.mu

    def noise(self):
        x = self.state
        dx = self.theta * (self.mu - x) + self.sigma * np.random.randn(len(x))
        self.state = x + dx
        return self.state

In [15]:
from tqdm import tqdm_notebook

In [16]:
for i in tqdm_notebook(range(MAX_EPISODES)):
        start = time()
        ep_rewards = []
        ep_action_dist = []
        s = env.reset()

        exploration_noise = OUNoise(action_dim, OU_MU, OU_THETA, OU_SIGMA)

        while True:

            a = sess.run(actor, feed_dict={state: s[np.newaxis, :]})

            epsilon = np.exp(-i/TAU2)
            a += epsilon * exploration_noise.noise() / env.action_space.high

            a = np.clip(a, env.action_space.low, env.action_space.high)
            s2, r, terminal, info = env.step(a[0])
            ep_action_dist.append(a[0])

            buffer.append((np.hstack(s), np.hstack(a), r, terminal, np.hstack(s2)))

            if len(buffer) > MINIBATCH_SIZE:
                s_batch, a_batch, r_batch, t_batch, s2_batch = sample_batch(MINIBATCH_SIZE)

                target_a = sess.run(actor_target, feed_dict={state: s2_batch})
                target_q = sess.run(critic_target, feed_dict={state: s2_batch, action: target_a})
                y = r_batch + GAMMA * target_q * ~t_batch


                sess.run([train_critic, train_actor], feed_dict={state: s_batch, action: a_batch, y_t: y})
                sess.run([update_actor, update_critic])

            s = s2
            ep_rewards.append(r)

            if terminal:
                print('Episode', i, '\tReward: %.2f' % np.sum(ep_rewards), '\tTime: %.1fs' % (time() - start),
                      '\tEpsilon: %.3f' % epsilon)
                exploration_noise.reset()
                break

Episode 0 	Reward: -106.77 	Time: 0.2s 	Epsilon: 1.000
Episode 1 	Reward: -106.18 	Time: 0.6s 	Epsilon: 0.961
Episode 2 	Reward: -132.41 	Time: 3.2s 	Epsilon: 0.923
Episode 3 	Reward: -112.71 	Time: 0.9s 	Epsilon: 0.887
Episode 4 	Reward: -110.05 	Time: 1.0s 	Epsilon: 0.852
Episode 5 	Reward: -117.50 	Time: 0.7s 	Epsilon: 0.819
Episode 6 	Reward: -120.88 	Time: 0.9s 	Epsilon: 0.787
Episode 7 	Reward: -114.14 	Time: 1.6s 	Epsilon: 0.756
Episode 8 	Reward: -149.60 	Time: 3.9s 	Epsilon: 0.726
Episode 9 	Reward: -153.57 	Time: 4.6s 	Epsilon: 0.698
Episode 10 	Reward: -213.52 	Time: 9.7s 	Epsilon: 0.670
Episode 11 	Reward: -104.18 	Time: 0.4s 	Epsilon: 0.644
Episode 12 	Reward: -168.95 	Time: 13.0s 	Epsilon: 0.619
Episode 13 	Reward: -102.12 	Time: 0.5s 	Epsilon: 0.595
Episode 14 	Reward: -104.10 	Time: 0.4s 	Epsilon: 0.571
Episode 15 	Reward: -125.06 	Time: 1.3s 	Epsilon: 0.549
Episode 16 	Reward: -102.89 	Time: 0.4s 	Epsilon: 0.527
Episode 17 	Reward: -102.60 	Time: 0.4s 	Epsilon: 0.507
E

Episode 146 	Reward: -119.55 	Time: 0.7s 	Epsilon: 0.003
Episode 147 	Reward: -119.55 	Time: 0.7s 	Epsilon: 0.003
Episode 148 	Reward: -119.46 	Time: 0.7s 	Epsilon: 0.003
Episode 149 	Reward: -119.60 	Time: 0.7s 	Epsilon: 0.003
Episode 150 	Reward: -119.57 	Time: 0.7s 	Epsilon: 0.002
Episode 151 	Reward: -118.35 	Time: 0.6s 	Epsilon: 0.002
Episode 152 	Reward: -119.42 	Time: 0.7s 	Epsilon: 0.002
Episode 153 	Reward: -118.32 	Time: 0.6s 	Epsilon: 0.002
Episode 154 	Reward: -118.36 	Time: 0.6s 	Epsilon: 0.002
Episode 155 	Reward: -118.35 	Time: 0.6s 	Epsilon: 0.002
Episode 156 	Reward: -118.36 	Time: 0.6s 	Epsilon: 0.002
Episode 157 	Reward: -118.44 	Time: 0.6s 	Epsilon: 0.002
Episode 158 	Reward: -119.41 	Time: 0.7s 	Epsilon: 0.002
Episode 159 	Reward: -118.25 	Time: 0.6s 	Epsilon: 0.002
Episode 160 	Reward: -118.39 	Time: 0.6s 	Epsilon: 0.002
Episode 161 	Reward: -118.23 	Time: 0.6s 	Epsilon: 0.002
Episode 162 	Reward: -118.28 	Time: 0.6s 	Epsilon: 0.002
Episode 163 	Reward: -118.19 	T

Episode 290 	Reward: -123.05 	Time: 0.4s 	Epsilon: 0.000
Episode 291 	Reward: -122.93 	Time: 0.4s 	Epsilon: 0.000
Episode 292 	Reward: -122.85 	Time: 0.4s 	Epsilon: 0.000
Episode 293 	Reward: -123.00 	Time: 0.4s 	Epsilon: 0.000
Episode 294 	Reward: -122.88 	Time: 0.4s 	Epsilon: 0.000
Episode 295 	Reward: -123.00 	Time: 0.4s 	Epsilon: 0.000
Episode 296 	Reward: -122.94 	Time: 0.4s 	Epsilon: 0.000
Episode 297 	Reward: -122.96 	Time: 0.4s 	Epsilon: 0.000
Episode 298 	Reward: -123.01 	Time: 0.4s 	Epsilon: 0.000
Episode 299 	Reward: -122.94 	Time: 0.4s 	Epsilon: 0.000
Episode 300 	Reward: -123.05 	Time: 0.4s 	Epsilon: 0.000
Episode 301 	Reward: -122.95 	Time: 0.4s 	Epsilon: 0.000
Episode 302 	Reward: -122.90 	Time: 0.4s 	Epsilon: 0.000
Episode 303 	Reward: -122.63 	Time: 0.4s 	Epsilon: 0.000
Episode 304 	Reward: -122.73 	Time: 0.4s 	Epsilon: 0.000
Episode 305 	Reward: -122.64 	Time: 0.4s 	Epsilon: 0.000
Episode 306 	Reward: -122.77 	Time: 0.4s 	Epsilon: 0.000
Episode 307 	Reward: -122.66 	T

Episode 434 	Reward: -114.10 	Time: 0.3s 	Epsilon: 0.000
Episode 435 	Reward: -112.80 	Time: 0.4s 	Epsilon: 0.000
Episode 436 	Reward: -114.50 	Time: 0.3s 	Epsilon: 0.000
Episode 437 	Reward: -114.60 	Time: 0.3s 	Epsilon: 0.000
Episode 438 	Reward: -114.11 	Time: 0.4s 	Epsilon: 0.000
Episode 439 	Reward: -113.82 	Time: 0.3s 	Epsilon: 0.000
Episode 440 	Reward: -114.11 	Time: 0.3s 	Epsilon: 0.000
Episode 441 	Reward: -114.12 	Time: 0.3s 	Epsilon: 0.000
Episode 442 	Reward: -114.43 	Time: 0.3s 	Epsilon: 0.000
Episode 443 	Reward: -113.80 	Time: 0.3s 	Epsilon: 0.000
Episode 444 	Reward: -113.92 	Time: 0.4s 	Epsilon: 0.000
Episode 445 	Reward: -112.17 	Time: 0.4s 	Epsilon: 0.000
Episode 446 	Reward: -112.29 	Time: 0.3s 	Epsilon: 0.000
Episode 447 	Reward: -109.92 	Time: 0.5s 	Epsilon: 0.000
Episode 448 	Reward: -114.22 	Time: 0.3s 	Epsilon: 0.000
Episode 449 	Reward: -114.29 	Time: 0.3s 	Epsilon: 0.000
Episode 450 	Reward: -114.26 	Time: 0.3s 	Epsilon: 0.000
Episode 451 	Reward: -114.30 	T

Episode 578 	Reward: -132.16 	Time: 1.1s 	Epsilon: 0.000
Episode 579 	Reward: -127.26 	Time: 0.8s 	Epsilon: 0.000
Episode 580 	Reward: -127.65 	Time: 0.8s 	Epsilon: 0.000
Episode 581 	Reward: -125.91 	Time: 0.7s 	Epsilon: 0.000
Episode 582 	Reward: -107.59 	Time: 0.3s 	Epsilon: 0.000
Episode 583 	Reward: -107.47 	Time: 0.3s 	Epsilon: 0.000
Episode 584 	Reward: -107.49 	Time: 0.3s 	Epsilon: 0.000
Episode 585 	Reward: -107.45 	Time: 0.3s 	Epsilon: 0.000
Episode 586 	Reward: -110.37 	Time: 0.6s 	Epsilon: 0.000
Episode 587 	Reward: -107.65 	Time: 0.3s 	Epsilon: 0.000
Episode 588 	Reward: -107.70 	Time: 0.3s 	Epsilon: 0.000
Episode 589 	Reward: -107.43 	Time: 0.3s 	Epsilon: 0.000
Episode 590 	Reward: -107.51 	Time: 0.3s 	Epsilon: 0.000
Episode 591 	Reward: -107.34 	Time: 0.3s 	Epsilon: 0.000
Episode 592 	Reward: -107.35 	Time: 0.3s 	Epsilon: 0.000
Episode 593 	Reward: -107.62 	Time: 0.3s 	Epsilon: 0.000
Episode 594 	Reward: -107.60 	Time: 0.3s 	Epsilon: 0.000
Episode 595 	Reward: -107.55 	T

Episode 722 	Reward: -111.33 	Time: 0.4s 	Epsilon: 0.000
Episode 723 	Reward: -108.61 	Time: 0.4s 	Epsilon: 0.000
Episode 724 	Reward: -109.19 	Time: 0.3s 	Epsilon: 0.000
Episode 725 	Reward: -109.19 	Time: 0.4s 	Epsilon: 0.000
Episode 726 	Reward: -108.21 	Time: 0.4s 	Epsilon: 0.000
Episode 727 	Reward: -111.78 	Time: 0.4s 	Epsilon: 0.000
Episode 728 	Reward: -109.04 	Time: 0.3s 	Epsilon: 0.000
Episode 729 	Reward: -108.58 	Time: 0.3s 	Epsilon: 0.000
Episode 730 	Reward: -109.77 	Time: 0.3s 	Epsilon: 0.000
Episode 731 	Reward: -108.51 	Time: 0.3s 	Epsilon: 0.000
Episode 732 	Reward: -108.38 	Time: 0.3s 	Epsilon: 0.000
Episode 733 	Reward: -107.77 	Time: 0.3s 	Epsilon: 0.000
Episode 734 	Reward: -110.02 	Time: 0.4s 	Epsilon: 0.000
Episode 735 	Reward: -111.14 	Time: 0.4s 	Epsilon: 0.000
Episode 736 	Reward: -108.73 	Time: 0.3s 	Epsilon: 0.000
Episode 737 	Reward: -111.30 	Time: 0.4s 	Epsilon: 0.000
Episode 738 	Reward: -107.85 	Time: 0.3s 	Epsilon: 0.000
Episode 739 	Reward: -109.84 	T

Episode 866 	Reward: -178.76 	Time: 13.7s 	Epsilon: 0.000
Episode 867 	Reward: -178.43 	Time: 13.6s 	Epsilon: 0.000
Episode 868 	Reward: -178.34 	Time: 13.6s 	Epsilon: 0.000
Episode 869 	Reward: -178.47 	Time: 13.6s 	Epsilon: 0.000
Episode 870 	Reward: -175.12 	Time: 6.7s 	Epsilon: 0.000
Episode 871 	Reward: -175.85 	Time: 13.7s 	Epsilon: 0.000
Episode 872 	Reward: -122.40 	Time: 2.8s 	Epsilon: 0.000
Episode 873 	Reward: -156.34 	Time: 13.7s 	Epsilon: 0.000
Episode 874 	Reward: -181.51 	Time: 13.8s 	Epsilon: 0.000
Episode 875 	Reward: -184.29 	Time: 13.9s 	Epsilon: 0.000
Episode 876 	Reward: -108.85 	Time: 0.6s 	Epsilon: 0.000
Episode 877 	Reward: -179.82 	Time: 13.9s 	Epsilon: 0.000
Episode 878 	Reward: -173.44 	Time: 13.8s 	Epsilon: 0.000
Episode 879 	Reward: -180.95 	Time: 13.9s 	Epsilon: 0.000
Episode 880 	Reward: -181.10 	Time: 13.9s 	Epsilon: 0.000
Episode 881 	Reward: -181.12 	Time: 13.9s 	Epsilon: 0.000
Episode 882 	Reward: -181.11 	Time: 13.9s 	Epsilon: 0.000
Episode 883 	Rewa

In [17]:
for i in tqdm_notebook(range(MAX_EPISODES)):
        start = time()
        ep_rewards = []
        ep_action_dist = []
        s = env.reset()

        exploration_noise = OUNoise(action_dim, OU_MU, OU_THETA, OU_SIGMA)

        while True:
            a = sess.run(actor, feed_dict={state: s[np.newaxis, :]})

            epsilon = np.exp(-i/TAU2)
            a += epsilon * exploration_noise.noise() / env.action_space.high

            a = np.clip(a, env.action_space.low, env.action_space.high)
            s2, r, terminal, info = env.step(a[0])
            ep_action_dist.append(a[0])

            buffer.append((np.hstack(s), np.hstack(a), r, terminal, np.hstack(s2)))

            if len(buffer) > MINIBATCH_SIZE:
                s_batch, a_batch, r_batch, t_batch, s2_batch = sample_batch(MINIBATCH_SIZE)

                target_a = sess.run(actor_target, feed_dict={state: s2_batch})
                target_q = sess.run(critic_target, feed_dict={state: s2_batch, action: target_a})
                y = r_batch + GAMMA * target_q * ~t_batch

                sess.run([train_critic, train_actor], feed_dict={state: s_batch, action: a_batch, y_t: y})
                sess.run([update_actor, update_critic])

            s = s2 
            ep_rewards.append(r)

            if terminal:
                print('Episode', i, '\tReward: %.2f' % np.sum(ep_rewards), '\tTime: %.1fs' % (time() - start),
                      '\tEpsilon: %.3f' % epsilon)
                exploration_noise.reset()
                break

Episode 0 	Reward: -123.10 	Time: 1.6s 	Epsilon: 1.000
Episode 1 	Reward: -124.12 	Time: 0.9s 	Epsilon: 0.961
Episode 2 	Reward: -124.42 	Time: 0.9s 	Epsilon: 0.923
Episode 3 	Reward: -125.43 	Time: 1.0s 	Epsilon: 0.887
Episode 4 	Reward: -122.69 	Time: 0.8s 	Epsilon: 0.852
Episode 5 	Reward: -122.38 	Time: 0.8s 	Epsilon: 0.819
Episode 6 	Reward: -129.86 	Time: 1.3s 	Epsilon: 0.787
Episode 7 	Reward: -122.90 	Time: 0.8s 	Epsilon: 0.756
Episode 8 	Reward: -126.62 	Time: 1.0s 	Epsilon: 0.726
Episode 9 	Reward: -120.53 	Time: 0.6s 	Epsilon: 0.698
Episode 10 	Reward: -129.10 	Time: 1.2s 	Epsilon: 0.670
Episode 11 	Reward: -122.15 	Time: 1.2s 	Epsilon: 0.644
Episode 12 	Reward: -123.93 	Time: 0.8s 	Epsilon: 0.619
Episode 13 	Reward: -121.13 	Time: 2.9s 	Epsilon: 0.595
Episode 14 	Reward: -140.55 	Time: 1.6s 	Epsilon: 0.571
Episode 15 	Reward: -124.76 	Time: 0.8s 	Epsilon: 0.549
Episode 16 	Reward: -137.46 	Time: 1.4s 	Epsilon: 0.527
Episode 17 	Reward: -121.46 	Time: 0.8s 	Epsilon: 0.507
Ep

Episode 164 	Reward: -197.65 	Time: 7.6s 	Epsilon: 0.001
Episode 165 	Reward: -151.46 	Time: 16.3s 	Epsilon: 0.001
Episode 166 	Reward: -146.56 	Time: 16.7s 	Epsilon: 0.001
Episode 167 	Reward: -137.26 	Time: 16.5s 	Epsilon: 0.001
Episode 168 	Reward: -145.91 	Time: 16.6s 	Epsilon: 0.001
Episode 169 	Reward: -159.61 	Time: 16.7s 	Epsilon: 0.001
Episode 170 	Reward: -166.29 	Time: 16.7s 	Epsilon: 0.001
Episode 171 	Reward: -169.29 	Time: 16.8s 	Epsilon: 0.001
Episode 172 	Reward: -168.42 	Time: 16.8s 	Epsilon: 0.001
Episode 173 	Reward: -155.60 	Time: 17.0s 	Epsilon: 0.001
Episode 174 	Reward: -168.43 	Time: 16.9s 	Epsilon: 0.001
Episode 175 	Reward: -147.42 	Time: 16.9s 	Epsilon: 0.001
Episode 176 	Reward: -145.79 	Time: 16.9s 	Epsilon: 0.001
Episode 177 	Reward: -138.87 	Time: 17.0s 	Epsilon: 0.001
Episode 178 	Reward: -127.47 	Time: 17.0s 	Epsilon: 0.001
Episode 179 	Reward: -123.29 	Time: 1.1s 	Epsilon: 0.001
Episode 180 	Reward: -126.82 	Time: 17.1s 	Epsilon: 0.001
Episode 181 	Rew

Episode 307 	Reward: -212.48 	Time: 14.0s 	Epsilon: 0.000
Episode 308 	Reward: -128.87 	Time: 19.2s 	Epsilon: 0.000
Episode 309 	Reward: -238.27 	Time: 18.7s 	Epsilon: 0.000
Episode 310 	Reward: -135.88 	Time: 2.9s 	Epsilon: 0.000
Episode 311 	Reward: -161.56 	Time: 4.6s 	Epsilon: 0.000
Episode 312 	Reward: -195.40 	Time: 11.3s 	Epsilon: 0.000
Episode 313 	Reward: -112.76 	Time: 4.1s 	Epsilon: 0.000
Episode 314 	Reward: -217.24 	Time: 14.1s 	Epsilon: 0.000
Episode 315 	Reward: -147.18 	Time: 19.3s 	Epsilon: 0.000
Episode 316 	Reward: -155.72 	Time: 7.0s 	Epsilon: 0.000
Episode 317 	Reward: -173.69 	Time: 19.4s 	Epsilon: 0.000
Episode 318 	Reward: -148.43 	Time: 8.0s 	Epsilon: 0.000
Episode 319 	Reward: -127.46 	Time: 3.8s 	Epsilon: 0.000
Episode 320 	Reward: -111.66 	Time: 3.0s 	Epsilon: 0.000
Episode 321 	Reward: -113.57 	Time: 1.0s 	Epsilon: 0.000
Episode 322 	Reward: -129.04 	Time: 19.4s 	Epsilon: 0.000
Episode 323 	Reward: -177.57 	Time: 6.0s 	Epsilon: 0.000
Episode 324 	Reward: -1

Episode 452 	Reward: 37.95 	Time: 22.0s 	Epsilon: 0.000
Episode 453 	Reward: -94.50 	Time: 2.4s 	Epsilon: 0.000
Episode 454 	Reward: 18.24 	Time: 21.7s 	Epsilon: 0.000
Episode 455 	Reward: -11.82 	Time: 22.0s 	Epsilon: 0.000
Episode 456 	Reward: -68.17 	Time: 22.0s 	Epsilon: 0.000
Episode 457 	Reward: 7.48 	Time: 22.0s 	Epsilon: 0.000
Episode 458 	Reward: -117.99 	Time: 22.0s 	Epsilon: 0.000
Episode 459 	Reward: -63.41 	Time: 11.1s 	Epsilon: 0.000
Episode 460 	Reward: -0.85 	Time: 22.1s 	Epsilon: 0.000
Episode 461 	Reward: -124.31 	Time: 22.2s 	Epsilon: 0.000
Episode 462 	Reward: -118.75 	Time: 22.2s 	Epsilon: 0.000
Episode 463 	Reward: -141.81 	Time: 22.2s 	Epsilon: 0.000
Episode 464 	Reward: 37.65 	Time: 22.4s 	Epsilon: 0.000
Episode 465 	Reward: -159.75 	Time: 8.2s 	Epsilon: 0.000
Episode 466 	Reward: -111.23 	Time: 5.5s 	Epsilon: 0.000
Episode 467 	Reward: -94.20 	Time: 22.4s 	Epsilon: 0.000
Episode 468 	Reward: -107.27 	Time: 22.4s 	Epsilon: 0.000
Episode 469 	Reward: 22.51 	Time:

Episode 626 	Reward: -35.88 	Time: 7.7s 	Epsilon: 0.000
Episode 627 	Reward: 242.16 	Time: 18.0s 	Epsilon: 0.000
Episode 628 	Reward: -16.03 	Time: 6.9s 	Epsilon: 0.000
Episode 629 	Reward: -66.52 	Time: 3.8s 	Epsilon: 0.000
Episode 630 	Reward: -49.63 	Time: 4.5s 	Epsilon: 0.000
Episode 631 	Reward: 107.26 	Time: 16.8s 	Epsilon: 0.000
Episode 632 	Reward: -66.62 	Time: 4.0s 	Epsilon: 0.000
Episode 633 	Reward: 240.18 	Time: 18.5s 	Epsilon: 0.000
Episode 634 	Reward: 65.14 	Time: 17.2s 	Epsilon: 0.000
Episode 635 	Reward: 228.07 	Time: 20.5s 	Epsilon: 0.000
Episode 636 	Reward: 87.30 	Time: 16.8s 	Epsilon: 0.000
Episode 637 	Reward: 218.38 	Time: 21.4s 	Epsilon: 0.000
Episode 638 	Reward: 217.90 	Time: 21.8s 	Epsilon: 0.000
Episode 639 	Reward: 214.14 	Time: 22.1s 	Epsilon: 0.000
Episode 640 	Reward: 213.81 	Time: 22.4s 	Epsilon: 0.000
Episode 641 	Reward: 71.92 	Time: 18.1s 	Epsilon: 0.000
Episode 642 	Reward: -49.38 	Time: 5.5s 	Epsilon: 0.000
Episode 643 	Reward: 219.30 	Time: 21.8s

Episode 771 	Reward: -117.96 	Time: 2.5s 	Epsilon: 0.000
Episode 772 	Reward: -120.64 	Time: 2.5s 	Epsilon: 0.000
Episode 773 	Reward: -124.39 	Time: 2.4s 	Epsilon: 0.000
Episode 774 	Reward: -119.99 	Time: 2.0s 	Epsilon: 0.000
Episode 775 	Reward: -121.58 	Time: 2.1s 	Epsilon: 0.000
Episode 776 	Reward: -113.43 	Time: 1.0s 	Epsilon: 0.000
Episode 777 	Reward: -120.65 	Time: 2.2s 	Epsilon: 0.000
Episode 778 	Reward: -120.77 	Time: 2.5s 	Epsilon: 0.000
Episode 779 	Reward: -124.66 	Time: 4.1s 	Epsilon: 0.000
Episode 780 	Reward: -119.18 	Time: 2.6s 	Epsilon: 0.000
Episode 781 	Reward: -126.94 	Time: 2.8s 	Epsilon: 0.000
Episode 782 	Reward: -121.44 	Time: 2.2s 	Epsilon: 0.000
Episode 783 	Reward: -127.53 	Time: 3.5s 	Epsilon: 0.000
Episode 784 	Reward: -119.64 	Time: 2.3s 	Epsilon: 0.000
Episode 785 	Reward: -116.81 	Time: 1.7s 	Epsilon: 0.000
Episode 786 	Reward: -115.30 	Time: 1.7s 	Epsilon: 0.000
Episode 787 	Reward: -114.40 	Time: 1.6s 	Epsilon: 0.000
Episode 788 	Reward: -126.27 	T

Episode 915 	Reward: -126.53 	Time: 3.7s 	Epsilon: 0.000
Episode 916 	Reward: -142.59 	Time: 7.7s 	Epsilon: 0.000
Episode 917 	Reward: -164.35 	Time: 9.1s 	Epsilon: 0.000
Episode 918 	Reward: -182.99 	Time: 13.7s 	Epsilon: 0.000
Episode 919 	Reward: -129.93 	Time: 3.4s 	Epsilon: 0.000
Episode 920 	Reward: -125.29 	Time: 2.1s 	Epsilon: 0.000
Episode 921 	Reward: -163.50 	Time: 12.2s 	Epsilon: 0.000
Episode 922 	Reward: -186.42 	Time: 14.4s 	Epsilon: 0.000
Episode 923 	Reward: -136.77 	Time: 6.8s 	Epsilon: 0.000
Episode 924 	Reward: -128.90 	Time: 27.3s 	Epsilon: 0.000
Episode 925 	Reward: -185.58 	Time: 10.9s 	Epsilon: 0.000
Episode 926 	Reward: -249.08 	Time: 21.8s 	Epsilon: 0.000
Episode 927 	Reward: -167.88 	Time: 8.1s 	Epsilon: 0.000
Episode 928 	Reward: -162.03 	Time: 6.8s 	Epsilon: 0.000
Episode 929 	Reward: -135.85 	Time: 4.9s 	Epsilon: 0.000
Episode 930 	Reward: -172.63 	Time: 8.4s 	Epsilon: 0.000
Episode 931 	Reward: -133.48 	Time: 4.3s 	Epsilon: 0.000
Episode 932 	Reward: -160

In [18]:
for i in tqdm_notebook(range(4000)):
        start = time()
        ep_rewards = []
        ep_action_dist = []
        s = env.reset()

        exploration_noise = OUNoise(action_dim, OU_MU, OU_THETA, OU_SIGMA)

        while True:

            a = sess.run(actor, feed_dict={state: s[np.newaxis, :]})

            epsilon = np.exp(-i/TAU2)
            a += epsilon * exploration_noise.noise() / env.action_space.high

            a = np.clip(a, env.action_space.low, env.action_space.high)
            s2, r, terminal, info = env.step(a[0])
            ep_action_dist.append(a[0])

            buffer.append((np.hstack(s), np.hstack(a), r, terminal, np.hstack(s2)))

            if len(buffer) > MINIBATCH_SIZE:
                s_batch, a_batch, r_batch, t_batch, s2_batch = sample_batch(MINIBATCH_SIZE)

                target_a = sess.run(actor_target, feed_dict={state: s2_batch})
                target_q = sess.run(critic_target, feed_dict={state: s2_batch, action: target_a})
                y = r_batch + GAMMA * target_q * ~t_batch

                sess.run([train_critic, train_actor], feed_dict={state: s_batch, action: a_batch, y_t: y})
                sess.run([update_actor, update_critic])

            s = s2 
            ep_rewards.append(r)

            if terminal:
                print('Episode', i, '\tReward: %.2f' % np.sum(ep_rewards), '\tTime: %.1fs' % (time() - start),
                      '\tEpsilon: %.3f' % epsilon)
                exploration_noise.reset()
                break

Episode 0 	Reward: -117.21 	Time: 1.5s 	Epsilon: 1.000
Episode 1 	Reward: -111.86 	Time: 1.1s 	Epsilon: 0.961
Episode 2 	Reward: -124.13 	Time: 6.3s 	Epsilon: 0.923
Episode 3 	Reward: -113.58 	Time: 1.0s 	Epsilon: 0.887
Episode 4 	Reward: -101.83 	Time: 2.2s 	Epsilon: 0.852
Episode 5 	Reward: -113.48 	Time: 1.0s 	Epsilon: 0.819
Episode 6 	Reward: -118.63 	Time: 1.3s 	Epsilon: 0.787
Episode 7 	Reward: -118.56 	Time: 1.4s 	Epsilon: 0.756
Episode 8 	Reward: -116.83 	Time: 1.0s 	Epsilon: 0.726
Episode 9 	Reward: -123.03 	Time: 1.2s 	Epsilon: 0.698
Episode 10 	Reward: -172.06 	Time: 28.6s 	Epsilon: 0.670
Episode 11 	Reward: -175.73 	Time: 28.5s 	Epsilon: 0.644
Episode 12 	Reward: -113.93 	Time: 0.9s 	Epsilon: 0.619
Episode 13 	Reward: -120.94 	Time: 5.5s 	Epsilon: 0.595
Episode 14 	Reward: -176.22 	Time: 28.8s 	Epsilon: 0.571
Episode 15 	Reward: -175.83 	Time: 28.6s 	Epsilon: 0.549
Episode 16 	Reward: -175.49 	Time: 28.6s 	Epsilon: 0.527
Episode 17 	Reward: -177.00 	Time: 28.7s 	Epsilon: 0.

Episode 175 	Reward: -19.52 	Time: 30.6s 	Epsilon: 0.001
Episode 176 	Reward: -57.93 	Time: 30.8s 	Epsilon: 0.001
Episode 177 	Reward: -58.73 	Time: 30.8s 	Epsilon: 0.001
Episode 178 	Reward: -82.58 	Time: 6.8s 	Epsilon: 0.001
Episode 179 	Reward: -128.28 	Time: 14.5s 	Epsilon: 0.001
Episode 180 	Reward: -166.15 	Time: 30.5s 	Epsilon: 0.001
Episode 181 	Reward: 77.19 	Time: 30.8s 	Epsilon: 0.001
Episode 182 	Reward: -70.52 	Time: 30.8s 	Epsilon: 0.001
Episode 183 	Reward: -194.39 	Time: 26.8s 	Epsilon: 0.001
Episode 184 	Reward: -251.94 	Time: 27.0s 	Epsilon: 0.001
Episode 185 	Reward: -66.05 	Time: 31.0s 	Epsilon: 0.001
Episode 186 	Reward: -170.85 	Time: 20.1s 	Epsilon: 0.001
Episode 187 	Reward: -120.76 	Time: 30.2s 	Epsilon: 0.001
Episode 188 	Reward: -55.83 	Time: 30.9s 	Epsilon: 0.001
Episode 189 	Reward: -65.79 	Time: 30.9s 	Epsilon: 0.001
Episode 190 	Reward: -129.03 	Time: 30.9s 	Epsilon: 0.001
Episode 191 	Reward: -57.11 	Time: 31.0s 	Epsilon: 0.000
Episode 192 	Reward: 13.80

Episode 323 	Reward: -140.42 	Time: 3.4s 	Epsilon: 0.000
Episode 324 	Reward: -12.59 	Time: 10.2s 	Epsilon: 0.000
Episode 325 	Reward: -136.17 	Time: 2.8s 	Epsilon: 0.000
Episode 326 	Reward: -155.36 	Time: 6.7s 	Epsilon: 0.000
Episode 327 	Reward: -137.93 	Time: 3.3s 	Epsilon: 0.000
Episode 328 	Reward: -184.13 	Time: 11.4s 	Epsilon: 0.000
Episode 329 	Reward: -71.93 	Time: 8.6s 	Epsilon: 0.000
Episode 330 	Reward: -147.13 	Time: 3.3s 	Epsilon: 0.000
Episode 331 	Reward: -144.57 	Time: 3.2s 	Epsilon: 0.000
Episode 332 	Reward: -97.38 	Time: 31.4s 	Epsilon: 0.000
Episode 333 	Reward: 217.01 	Time: 26.9s 	Epsilon: 0.000
Episode 334 	Reward: -116.24 	Time: 7.6s 	Epsilon: 0.000
Episode 335 	Reward: -151.72 	Time: 6.1s 	Epsilon: 0.000
Episode 336 	Reward: -127.48 	Time: 1.4s 	Epsilon: 0.000
Episode 337 	Reward: -127.02 	Time: 2.1s 	Epsilon: 0.000
Episode 338 	Reward: -127.81 	Time: 1.6s 	Epsilon: 0.000
Episode 339 	Reward: -115.35 	Time: 6.7s 	Epsilon: 0.000
Episode 340 	Reward: -99.64 	Ti

Episode 467 	Reward: -63.19 	Time: 4.7s 	Epsilon: 0.000
Episode 468 	Reward: 200.52 	Time: 30.2s 	Epsilon: 0.000
Episode 469 	Reward: 24.82 	Time: 14.9s 	Epsilon: 0.000
Episode 470 	Reward: -57.06 	Time: 5.3s 	Epsilon: 0.000
Episode 471 	Reward: -65.81 	Time: 5.8s 	Epsilon: 0.000
Episode 472 	Reward: 209.55 	Time: 28.9s 	Epsilon: 0.000
Episode 473 	Reward: -106.49 	Time: 1.5s 	Epsilon: 0.000
Episode 474 	Reward: 28.10 	Time: 18.2s 	Epsilon: 0.000
Episode 475 	Reward: 223.92 	Time: 26.0s 	Epsilon: 0.000
Episode 476 	Reward: 39.28 	Time: 18.4s 	Epsilon: 0.000
Episode 477 	Reward: -16.08 	Time: 15.6s 	Epsilon: 0.000
Episode 478 	Reward: -41.44 	Time: 7.9s 	Epsilon: 0.000
Episode 479 	Reward: 201.20 	Time: 30.3s 	Epsilon: 0.000
Episode 480 	Reward: -110.17 	Time: 3.1s 	Epsilon: 0.000
Episode 481 	Reward: -8.15 	Time: 13.0s 	Epsilon: 0.000
Episode 482 	Reward: 188.36 	Time: 31.2s 	Epsilon: 0.000
Episode 483 	Reward: 176.82 	Time: 31.3s 	Epsilon: 0.000
Episode 484 	Reward: 23.25 	Time: 19.1s

Episode 612 	Reward: 108.02 	Time: 31.3s 	Epsilon: 0.000
Episode 613 	Reward: 52.20 	Time: 31.3s 	Epsilon: 0.000
Episode 614 	Reward: 201.86 	Time: 31.1s 	Epsilon: 0.000
Episode 615 	Reward: 186.82 	Time: 31.3s 	Epsilon: 0.000
Episode 616 	Reward: 197.03 	Time: 31.4s 	Epsilon: 0.000
Episode 617 	Reward: -115.76 	Time: 1.5s 	Epsilon: 0.000
Episode 618 	Reward: 151.97 	Time: 31.2s 	Epsilon: 0.000
Episode 619 	Reward: 167.25 	Time: 31.3s 	Epsilon: 0.000
Episode 620 	Reward: 93.85 	Time: 31.3s 	Epsilon: 0.000
Episode 621 	Reward: 179.96 	Time: 31.4s 	Epsilon: 0.000
Episode 622 	Reward: 103.40 	Time: 31.2s 	Epsilon: 0.000
Episode 623 	Reward: -167.83 	Time: 14.0s 	Epsilon: 0.000
Episode 624 	Reward: -127.19 	Time: 26.1s 	Epsilon: 0.000
Episode 625 	Reward: -140.89 	Time: 21.9s 	Epsilon: 0.000
Episode 626 	Reward: 70.74 	Time: 31.3s 	Epsilon: 0.000
Episode 627 	Reward: -165.24 	Time: 30.5s 	Epsilon: 0.000
Episode 628 	Reward: -147.53 	Time: 7.3s 	Epsilon: 0.000
Episode 629 	Reward: -45.06 	T

Episode 761 	Reward: -166.43 	Time: 8.2s 	Epsilon: 0.000
Episode 762 	Reward: -152.64 	Time: 5.9s 	Epsilon: 0.000
Episode 763 	Reward: -135.43 	Time: 4.0s 	Epsilon: 0.000
Episode 764 	Reward: -133.31 	Time: 4.3s 	Epsilon: 0.000
Episode 765 	Reward: -175.29 	Time: 9.7s 	Epsilon: 0.000
Episode 766 	Reward: -142.64 	Time: 6.1s 	Epsilon: 0.000
Episode 767 	Reward: -126.65 	Time: 0.9s 	Epsilon: 0.000
Episode 768 	Reward: -142.03 	Time: 5.6s 	Epsilon: 0.000
Episode 769 	Reward: -124.92 	Time: 0.9s 	Epsilon: 0.000
Episode 770 	Reward: -125.30 	Time: 0.9s 	Epsilon: 0.000
Episode 771 	Reward: -155.39 	Time: 8.0s 	Epsilon: 0.000
Episode 772 	Reward: -112.91 	Time: 5.6s 	Epsilon: 0.000
Episode 773 	Reward: -157.32 	Time: 6.5s 	Epsilon: 0.000
Episode 774 	Reward: -124.05 	Time: 0.9s 	Epsilon: 0.000
Episode 775 	Reward: -146.12 	Time: 7.5s 	Epsilon: 0.000
Episode 776 	Reward: -148.94 	Time: 7.3s 	Epsilon: 0.000
Episode 777 	Reward: -148.62 	Time: 6.0s 	Epsilon: 0.000
Episode 778 	Reward: -120.44 	T

KeyboardInterrupt: 

In [19]:
for i in tqdm_notebook(range(2000)):
        start = time()
        ep_rewards = []
        ep_action_dist = []
        s = env.reset()

        exploration_noise = OUNoise(action_dim, OU_MU, OU_THETA, OU_SIGMA)

        while True:

            a = sess.run(actor, feed_dict={state: s[np.newaxis, :]})

            epsilon = np.exp(-i/TAU2)
            a += epsilon * exploration_noise.noise() / env.action_space.high

            a = np.clip(a, env.action_space.low, env.action_space.high)
            s2, r, terminal, info = env.step(a[0])
            ep_action_dist.append(a[0])

            buffer.append((np.hstack(s), np.hstack(a), r, terminal, np.hstack(s2)))

            if len(buffer) > MINIBATCH_SIZE:
                s_batch, a_batch, r_batch, t_batch, s2_batch = sample_batch(MINIBATCH_SIZE)

                target_a = sess.run(actor_target, feed_dict={state: s2_batch})
                target_q = sess.run(critic_target, feed_dict={state: s2_batch, action: target_a})
                y = r_batch + GAMMA * target_q * ~t_batch

                sess.run([train_critic, train_actor], feed_dict={state: s_batch, action: a_batch, y_t: y})
                sess.run([update_actor, update_critic])

            s = s2
            ep_rewards.append(r)

            if terminal:
                print('Episode', i, '\tReward: %.2f' % np.sum(ep_rewards), '\tTime: %.1fs' % (time() - start),
                      '\tEpsilon: %.3f' % epsilon)
                exploration_noise.reset()
                break

Episode 0 	Reward: 1.27 	Time: 20.7s 	Epsilon: 1.000
Episode 1 	Reward: 195.28 	Time: 31.1s 	Epsilon: 0.961
Episode 7 	Reward: -107.72 	Time: 31.2s 	Epsilon: 0.756
Episode 8 	Reward: 103.29 	Time: 31.4s 	Epsilon: 0.726
Episode 9 	Reward: 222.84 	Time: 29.6s 	Epsilon: 0.698
Episode 10 	Reward: -77.39 	Time: 7.7s 	Epsilon: 0.670
Episode 11 	Reward: 218.48 	Time: 29.8s 	Epsilon: 0.644
Episode 12 	Reward: 41.50 	Time: 28.8s 	Epsilon: 0.619
Episode 13 	Reward: 75.95 	Time: 31.3s 	Epsilon: 0.595
Episode 14 	Reward: 83.85 	Time: 31.3s 	Epsilon: 0.571
Episode 15 	Reward: -25.17 	Time: 21.3s 	Epsilon: 0.549
Episode 16 	Reward: 62.88 	Time: 18.0s 	Epsilon: 0.527
Episode 17 	Reward: 170.93 	Time: 31.1s 	Epsilon: 0.507
Episode 18 	Reward: -148.17 	Time: 20.9s 	Epsilon: 0.487
Episode 19 	Reward: 44.84 	Time: 24.5s 	Epsilon: 0.468
Episode 20 	Reward: -50.13 	Time: 8.9s 	Epsilon: 0.449
Episode 21 	Reward: 207.84 	Time: 31.0s 	Epsilon: 0.432
Episode 22 	Reward: -68.94 	Time: 6.4s 	Epsilon: 0.415
Episo

Episode 151 	Reward: -109.55 	Time: 1.2s 	Epsilon: 0.002
Episode 152 	Reward: -111.15 	Time: 1.0s 	Epsilon: 0.002
Episode 153 	Reward: -113.11 	Time: 1.0s 	Epsilon: 0.002
Episode 154 	Reward: -112.19 	Time: 0.9s 	Epsilon: 0.002
Episode 155 	Reward: -111.35 	Time: 1.2s 	Epsilon: 0.002
Episode 156 	Reward: -108.93 	Time: 1.2s 	Epsilon: 0.002
Episode 157 	Reward: -113.19 	Time: 0.9s 	Epsilon: 0.002
Episode 158 	Reward: -112.66 	Time: 1.1s 	Epsilon: 0.002
Episode 159 	Reward: -112.31 	Time: 1.0s 	Epsilon: 0.002
Episode 160 	Reward: -114.34 	Time: 0.9s 	Epsilon: 0.002
Episode 161 	Reward: -108.03 	Time: 1.2s 	Epsilon: 0.002
Episode 162 	Reward: -114.08 	Time: 0.9s 	Epsilon: 0.002
Episode 163 	Reward: -112.32 	Time: 1.0s 	Epsilon: 0.001
Episode 164 	Reward: -114.21 	Time: 1.0s 	Epsilon: 0.001
Episode 165 	Reward: -114.24 	Time: 0.9s 	Epsilon: 0.001
Episode 166 	Reward: -111.59 	Time: 1.0s 	Epsilon: 0.001
Episode 167 	Reward: -109.87 	Time: 1.1s 	Epsilon: 0.001
Episode 168 	Reward: -112.99 	T

Episode 295 	Reward: -135.39 	Time: 3.3s 	Epsilon: 0.000
Episode 296 	Reward: -114.72 	Time: 0.9s 	Epsilon: 0.000
Episode 297 	Reward: -96.58 	Time: 6.7s 	Epsilon: 0.000
Episode 298 	Reward: -130.81 	Time: 2.9s 	Epsilon: 0.000
Episode 299 	Reward: -105.75 	Time: 2.8s 	Epsilon: 0.000
Episode 300 	Reward: -140.50 	Time: 2.7s 	Epsilon: 0.000
Episode 301 	Reward: -107.16 	Time: 4.6s 	Epsilon: 0.000
Episode 302 	Reward: -115.09 	Time: 1.0s 	Epsilon: 0.000
Episode 303 	Reward: -111.77 	Time: 1.4s 	Epsilon: 0.000
Episode 304 	Reward: -98.77 	Time: 2.8s 	Epsilon: 0.000
Episode 305 	Reward: -111.15 	Time: 1.4s 	Epsilon: 0.000
Episode 306 	Reward: -112.90 	Time: 9.8s 	Epsilon: 0.000
Episode 307 	Reward: -112.40 	Time: 1.7s 	Epsilon: 0.000
Episode 308 	Reward: -115.50 	Time: 1.0s 	Epsilon: 0.000
Episode 309 	Reward: -116.08 	Time: 0.9s 	Epsilon: 0.000
Episode 310 	Reward: -118.50 	Time: 1.1s 	Epsilon: 0.000
Episode 311 	Reward: -102.63 	Time: 5.7s 	Epsilon: 0.000
Episode 312 	Reward: -116.61 	Tim

Episode 439 	Reward: 0.78 	Time: 31.4s 	Epsilon: 0.000
Episode 440 	Reward: -173.50 	Time: 31.2s 	Epsilon: 0.000
Episode 441 	Reward: -21.71 	Time: 31.4s 	Epsilon: 0.000
Episode 442 	Reward: -149.63 	Time: 31.2s 	Epsilon: 0.000
Episode 443 	Reward: -171.52 	Time: 31.5s 	Epsilon: 0.000
Episode 444 	Reward: -172.23 	Time: 31.3s 	Epsilon: 0.000
Episode 445 	Reward: -170.43 	Time: 31.4s 	Epsilon: 0.000
Episode 446 	Reward: -172.08 	Time: 31.5s 	Epsilon: 0.000
Episode 447 	Reward: 55.48 	Time: 31.2s 	Epsilon: 0.000
Episode 448 	Reward: -169.32 	Time: 31.4s 	Epsilon: 0.000
Episode 449 	Reward: -167.23 	Time: 31.4s 	Epsilon: 0.000
Episode 450 	Reward: 108.46 	Time: 31.4s 	Epsilon: 0.000
Episode 451 	Reward: 128.47 	Time: 31.3s 	Epsilon: 0.000
Episode 452 	Reward: 76.98 	Time: 31.6s 	Epsilon: 0.000
Episode 453 	Reward: 79.65 	Time: 31.3s 	Epsilon: 0.000
Episode 454 	Reward: 50.59 	Time: 31.5s 	Epsilon: 0.000
Episode 455 	Reward: -42.47 	Time: 31.1s 	Epsilon: 0.000
Episode 456 	Reward: -43.49 	

Episode 584 	Reward: -87.32 	Time: 3.5s 	Epsilon: 0.000
Episode 585 	Reward: -109.94 	Time: 1.2s 	Epsilon: 0.000
Episode 586 	Reward: 252.19 	Time: 21.1s 	Epsilon: 0.000
Episode 587 	Reward: -15.90 	Time: 8.6s 	Epsilon: 0.000
Episode 588 	Reward: 233.69 	Time: 25.2s 	Epsilon: 0.000
Episode 589 	Reward: -49.36 	Time: 7.8s 	Epsilon: 0.000
Episode 590 	Reward: 225.51 	Time: 26.7s 	Epsilon: 0.000
Episode 591 	Reward: 223.12 	Time: 27.1s 	Epsilon: 0.000
Episode 592 	Reward: -81.70 	Time: 5.7s 	Epsilon: 0.000
Episode 593 	Reward: -17.85 	Time: 16.6s 	Epsilon: 0.000
Episode 594 	Reward: 239.71 	Time: 24.1s 	Epsilon: 0.000
Episode 595 	Reward: 72.14 	Time: 21.2s 	Epsilon: 0.000
Episode 596 	Reward: 216.97 	Time: 28.5s 	Epsilon: 0.000
Episode 597 	Reward: -48.11 	Time: 8.0s 	Epsilon: 0.000
Episode 598 	Reward: -7.72 	Time: 12.1s 	Epsilon: 0.000
Episode 599 	Reward: -182.60 	Time: 31.3s 	Epsilon: 0.000
Episode 600 	Reward: 235.48 	Time: 24.5s 	Epsilon: 0.000
Episode 601 	Reward: 29.29 	Time: 20.

Episode 729 	Reward: -134.56 	Time: 2.1s 	Epsilon: 0.000
Episode 730 	Reward: -133.77 	Time: 1.9s 	Epsilon: 0.000
Episode 731 	Reward: 235.64 	Time: 24.0s 	Epsilon: 0.000
Episode 732 	Reward: 74.41 	Time: 16.7s 	Epsilon: 0.000
Episode 733 	Reward: -91.50 	Time: 4.3s 	Epsilon: 0.000
Episode 734 	Reward: -130.34 	Time: 1.8s 	Epsilon: 0.000
Episode 735 	Reward: -135.80 	Time: 1.9s 	Epsilon: 0.000
Episode 736 	Reward: -24.90 	Time: 11.2s 	Epsilon: 0.000
Episode 737 	Reward: -25.40 	Time: 6.8s 	Epsilon: 0.000
Episode 738 	Reward: -156.06 	Time: 31.2s 	Epsilon: 0.000
Episode 739 	Reward: -77.49 	Time: 31.2s 	Epsilon: 0.000
Episode 740 	Reward: -148.51 	Time: 31.7s 	Epsilon: 0.000
Episode 741 	Reward: -161.03 	Time: 31.2s 	Epsilon: 0.000
Episode 742 	Reward: -154.39 	Time: 31.2s 	Epsilon: 0.000
Episode 743 	Reward: -152.11 	Time: 31.5s 	Epsilon: 0.000
Episode 744 	Reward: -129.42 	Time: 31.2s 	Epsilon: 0.000
Episode 745 	Reward: -130.73 	Time: 31.6s 	Epsilon: 0.000
Episode 746 	Reward: -145.4

Episode 873 	Reward: -145.60 	Time: 4.4s 	Epsilon: 0.000
Episode 874 	Reward: -148.22 	Time: 4.9s 	Epsilon: 0.000
Episode 875 	Reward: -158.42 	Time: 6.8s 	Epsilon: 0.000
Episode 876 	Reward: -170.26 	Time: 8.8s 	Epsilon: 0.000
Episode 877 	Reward: -150.73 	Time: 5.3s 	Epsilon: 0.000
Episode 878 	Reward: -180.87 	Time: 11.2s 	Epsilon: 0.000
Episode 879 	Reward: -158.37 	Time: 7.0s 	Epsilon: 0.000
Episode 880 	Reward: -172.89 	Time: 9.6s 	Epsilon: 0.000
Episode 881 	Reward: -189.96 	Time: 12.9s 	Epsilon: 0.000
Episode 882 	Reward: -158.89 	Time: 6.8s 	Epsilon: 0.000
Episode 883 	Reward: -161.20 	Time: 7.7s 	Epsilon: 0.000
Episode 884 	Reward: -196.36 	Time: 14.2s 	Epsilon: 0.000
Episode 885 	Reward: -175.22 	Time: 10.1s 	Epsilon: 0.000
Episode 886 	Reward: -164.18 	Time: 7.5s 	Epsilon: 0.000
Episode 887 	Reward: -260.11 	Time: 25.7s 	Epsilon: 0.000
Episode 888 	Reward: -178.06 	Time: 31.4s 	Epsilon: 0.000
Episode 889 	Reward: -172.41 	Time: 31.3s 	Epsilon: 0.000
Episode 890 	Reward: -18

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [20]:
for i in tqdm_notebook(range(500)):
        start = time()
        ep_rewards = []
        ep_action_dist = []
        s = env.reset()

        exploration_noise = OUNoise(action_dim, OU_MU, OU_THETA, OU_SIGMA)

        while True:

            a = sess.run(actor, feed_dict={state: s[np.newaxis, :]})

            epsilon = 0
            a += epsilon * exploration_noise.noise() / env.action_space.high

            a = np.clip(a, env.action_space.low, env.action_space.high)
            s2, r, terminal, info = env.step(a[0])
            ep_action_dist.append(a[0])

            buffer.append((np.hstack(s), np.hstack(a), r, terminal, np.hstack(s2)))

            if len(buffer) > MINIBATCH_SIZE:
                s_batch, a_batch, r_batch, t_batch, s2_batch = sample_batch(MINIBATCH_SIZE)

                target_a = sess.run(actor_target, feed_dict={state: s2_batch})
                target_q = sess.run(critic_target, feed_dict={state: s2_batch, action: target_a})
                y = r_batch + GAMMA * target_q * ~t_batch

                sess.run([train_critic, train_actor], feed_dict={state: s_batch, action: a_batch, y_t: y})
                sess.run([update_actor, update_critic])

            s = s2 
            ep_rewards.append(r)

            if terminal:
                print('Episode', i, '\tReward: %.2f' % np.sum(ep_rewards), '\tTime: %.1fs' % (time() - start),
                      '\tEpsilon: %.3f' % epsilon)
                exploration_noise.reset()
                break

Episode 0 	Reward: -101.39 	Time: 2.0s 	Epsilon: 0.000
Episode 1 	Reward: -104.01 	Time: 2.1s 	Epsilon: 0.000
Episode 2 	Reward: -78.70 	Time: 4.4s 	Epsilon: 0.000
Episode 3 	Reward: -108.22 	Time: 1.5s 	Epsilon: 0.000
Episode 4 	Reward: -87.94 	Time: 2.9s 	Epsilon: 0.000
Episode 5 	Reward: -60.50 	Time: 6.5s 	Epsilon: 0.000
Episode 6 	Reward: 93.57 	Time: 27.2s 	Epsilon: 0.000
Episode 7 	Reward: 216.59 	Time: 26.6s 	Epsilon: 0.000
Episode 8 	Reward: -84.70 	Time: 5.5s 	Epsilon: 0.000
Episode 9 	Reward: 4.01 	Time: 10.6s 	Epsilon: 0.000
Episode 10 	Reward: 241.81 	Time: 22.0s 	Epsilon: 0.000
Episode 11 	Reward: -51.50 	Time: 6.6s 	Epsilon: 0.000
Episode 12 	Reward: 247.18 	Time: 20.8s 	Epsilon: 0.000
Episode 13 	Reward: 238.20 	Time: 22.5s 	Epsilon: 0.000
Episode 14 	Reward: -7.09 	Time: 8.8s 	Epsilon: 0.000
Episode 15 	Reward: -109.98 	Time: 1.3s 	Epsilon: 0.000
Episode 16 	Reward: 237.01 	Time: 22.6s 	Epsilon: 0.000
Episode 17 	Reward: 232.36 	Time: 23.7s 	Epsilon: 0.000
Episode 18 	

Episode 148 	Reward: -97.49 	Time: 7.2s 	Epsilon: 0.000
Episode 149 	Reward: 107.65 	Time: 31.2s 	Epsilon: 0.000
Episode 150 	Reward: 76.82 	Time: 31.5s 	Epsilon: 0.000
Episode 151 	Reward: -89.60 	Time: 2.1s 	Epsilon: 0.000
Episode 152 	Reward: 81.06 	Time: 23.1s 	Epsilon: 0.000
Episode 153 	Reward: 95.12 	Time: 31.2s 	Epsilon: 0.000
Episode 154 	Reward: 88.51 	Time: 31.1s 	Epsilon: 0.000
Episode 155 	Reward: 25.31 	Time: 31.1s 	Epsilon: 0.000
Episode 156 	Reward: 88.63 	Time: 31.1s 	Epsilon: 0.000
Episode 157 	Reward: 26.51 	Time: 31.3s 	Epsilon: 0.000
Episode 158 	Reward: -4.32 	Time: 31.3s 	Epsilon: 0.000
Episode 159 	Reward: -80.47 	Time: 25.0s 	Epsilon: 0.000
Episode 160 	Reward: -98.63 	Time: 12.0s 	Epsilon: 0.000
Episode 161 	Reward: -70.01 	Time: 10.4s 	Epsilon: 0.000
Episode 162 	Reward: -26.08 	Time: 23.4s 	Epsilon: 0.000
Episode 163 	Reward: 81.36 	Time: 31.1s 	Epsilon: 0.000
Episode 164 	Reward: 231.57 	Time: 24.0s 	Epsilon: 0.000
Episode 165 	Reward: -51.80 	Time: 7.7s 	E

Episode 294 	Reward: -36.66 	Time: 7.7s 	Epsilon: 0.000
Episode 295 	Reward: -78.68 	Time: 4.5s 	Epsilon: 0.000
Episode 296 	Reward: -79.96 	Time: 4.8s 	Epsilon: 0.000
Episode 297 	Reward: -93.31 	Time: 3.4s 	Epsilon: 0.000
Episode 298 	Reward: 76.99 	Time: 29.5s 	Epsilon: 0.000
Episode 299 	Reward: -60.26 	Time: 5.6s 	Epsilon: 0.000
Episode 300 	Reward: -83.93 	Time: 2.9s 	Epsilon: 0.000
Episode 301 	Reward: 68.84 	Time: 16.5s 	Epsilon: 0.000
Episode 302 	Reward: -90.99 	Time: 2.6s 	Epsilon: 0.000
Episode 303 	Reward: -79.80 	Time: 3.6s 	Epsilon: 0.000
Episode 304 	Reward: -83.82 	Time: 3.6s 	Epsilon: 0.000
Episode 305 	Reward: -74.35 	Time: 5.3s 	Epsilon: 0.000
Episode 306 	Reward: -18.13 	Time: 13.1s 	Epsilon: 0.000
Episode 307 	Reward: 3.87 	Time: 16.2s 	Epsilon: 0.000
Episode 308 	Reward: -14.47 	Time: 8.9s 	Epsilon: 0.000
Episode 309 	Reward: -16.03 	Time: 21.0s 	Epsilon: 0.000
Episode 310 	Reward: -43.76 	Time: 6.5s 	Epsilon: 0.000
Episode 311 	Reward: -22.02 	Time: 9.9s 	Epsilo

Episode 464 	Reward: -87.92 	Time: 4.0s 	Epsilon: 0.000
Episode 465 	Reward: 259.63 	Time: 18.7s 	Epsilon: 0.000
Episode 466 	Reward: 260.75 	Time: 18.4s 	Epsilon: 0.000
Episode 467 	Reward: 264.46 	Time: 17.8s 	Epsilon: 0.000
Episode 468 	Reward: 257.41 	Time: 19.1s 	Epsilon: 0.000
Episode 469 	Reward: 245.98 	Time: 20.9s 	Epsilon: 0.000
Episode 470 	Reward: 246.67 	Time: 21.3s 	Epsilon: 0.000
Episode 471 	Reward: 255.37 	Time: 19.5s 	Epsilon: 0.000
Episode 472 	Reward: 251.91 	Time: 20.1s 	Epsilon: 0.000
Episode 473 	Reward: 243.69 	Time: 21.9s 	Epsilon: 0.000
Episode 474 	Reward: 258.36 	Time: 18.9s 	Epsilon: 0.000
Episode 475 	Reward: 258.35 	Time: 18.8s 	Epsilon: 0.000
Episode 476 	Reward: 255.06 	Time: 19.5s 	Epsilon: 0.000
Episode 477 	Reward: 251.94 	Time: 19.5s 	Epsilon: 0.000
Episode 478 	Reward: -69.71 	Time: 4.1s 	Epsilon: 0.000
Episode 479 	Reward: 263.82 	Time: 17.8s 	Epsilon: 0.000
Episode 480 	Reward: 15.08 	Time: 8.9s 	Epsilon: 0.000
Episode 481 	Reward: 12.83 	Time: 8

In [21]:
total_revs = []
for i in range(100):
    s = env.reset()
    reward = 0
    terminal = False
    while not terminal:
        a = sess.run(actor, feed_dict={state: s[np.newaxis, :]})
        a = np.clip(a, env.action_space.low, env.action_space.high)
        s2, r, terminal, info = env.step(a[0])
        reward += r
        s = s2
    print('Episode {}, reward {}'.format(i, reward))
    total_revs.append(reward)
print('Mean {}'.format(np.mean(total_revs)))

Episode 0, reward -53.840282578928495
Episode 1, reward -15.510818410469184
Episode 2, reward 262.9147307028609
Episode 3, reward -84.08878572282813
Episode 4, reward 56.92264955937105
Episode 5, reward 266.761057973557
Episode 6, reward 62.71023316838503
Episode 7, reward 262.80682897215655
Episode 8, reward 263.49897654778675
Episode 9, reward 262.42917934390556
Episode 10, reward 85.89160106166614
Episode 11, reward 262.24843150090976
Episode 12, reward -86.04997280291842
Episode 13, reward 266.5384467335321
Episode 14, reward 266.4375394157638
Episode 15, reward 262.4215871112785
Episode 16, reward 263.9088703930391
Episode 17, reward 27.50957828747849
Episode 18, reward 265.21481660584806
Episode 19, reward 265.3917795089199
Episode 20, reward 115.09157402795009
Episode 21, reward -12.399595859934209
Episode 22, reward 265.9485955806393
Episode 23, reward 266.27518252672036
Episode 24, reward 20.92691557877812
Episode 25, reward 2.193675628071176
Episode 26, reward 264.07345534944